In [8]:
import pandas as pd
import numpy as np
from collections import Counter, defaultdict
from sklearn.model_selection import train_test_split
from statsmodels.discrete.discrete_model import Logit
import sklearn.model_selection as cv
import sklearn.metrics as metrics
from statsmodels.tools import add_constant
from sklearn.metrics import roc_curve as sk_roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, VotingClassifier
import pymc3 as pm

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
df = pd.read_csv('data/train.csv')
df.head()

,amount,hour,account_id,field1,field2,flag1,field3,field4,indicator1,indicator2,flag2,flag4,flag5,fraud
0,12.95,0,1234567890123456,0,0,0,-723,19,0,0,0,0,1,0
1,38.85,0,1234567890123460,3,1,0,-4074,21,0,0,1,0,1,0
2,12.95,0,1234567890123461,3,0,1,-2753,24,0,0,1,0,1,0
3,10.36,0,1234567890123463,3,1,1,5927,7,0,0,1,0,2,0
4,49.95,0,1234567890123464,3,0,1,4942,9,0,0,1,0,1,0


In [18]:
y = df['fraud']
X = df.drop(columns='fraud')
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, random_state=7)

In [22]:
(y_train == 0).sum()

48076

In [24]:
train_matrix = np.array([[(y_train == 0).sum(),0], [0, (y_train == 1).sum()]])
test_matrix = np.array([[(y_test == 0).sum(),0], [0, (y_test == 1).sum()]])

In [46]:
train_matrix

array([[48076,     0],
       [    0,  1282]])

In [45]:
test_matrix

array([[16029,     0],
       [    0,   424]])

In [12]:
'''def model_scores(X, y, estimator, p=False, train=False):
    if train:
        scores = cv.cross_validate(estimator, X, y, scoring=['accuracy', 'precision', 'recall', 'neg_log_loss'], cv=5)
        acc = np.mean(scores['test_accuracy'])
        precision = np.mean(scores['test_precision'])
        recall = np.mean(scores['test_recall'])
        log_loss = -np.mean(scores['test_neg_log_loss'])
    else:
        y_hat = estimator.predict(X)
        acc = metrics.accuracy_score(y, y_hat)
        precision = metrics.precision_score(y, y_hat)
        recall = metrics.recall_score(y, y_hat)
        log_loss = metrics.log_loss(y, estimator.predict_proba(X))
    if p:
        print ("Accuracy: {0:2.3} | Precision: {1} | Recall: {2} | Log_loss: {3:2.3}".format(
                                                                    acc, precision, recall, log_loss))                                                           
    else:
        return acc, precision, recall, log_loss'''

In [42]:
LR = LogisticRegression(solver='lbfgs',max_iter=10000, tol=.000001)
LR.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=1e-06, verbose=0,
                   warm_start=False)

In [43]:
metrics.confusion_matrix(y_train, LR.predict(X_train))

array([[48076,     0],
       [ 1282,     0]])

In [64]:
(LR.predict_proba(X_train)[:,0] != .5).sum()

0

In [44]:
#WELL SHIT

In [47]:
RFC = RandomForestClassifier()
RFC.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [48]:
metrics.confusion_matrix(y_train, RFC.predict(X_train))

array([[48047,    29],
       [  361,   921]])

In [49]:
metrics.confusion_matrix(y_test, RFC.predict(X_test))

array([[15993,    36],
       [  422,     2]])

In [58]:
921/(921+29)

0.9694736842105263

In [52]:
metrics.recall_score(y_train, RFC.predict(X_train))

0.718408736349454

In [57]:
metrics.precision_score(y_train, RFC.predict(X_train))

0.9694736842105263